In [6]:
import numpy as np
import time

# Define the number of VMs and their capabilities (100 and 1000 MIPS)
vm_capabilities = [100, 1000]
num_vms = 10  # Number of VMs per edge server

# Define tasks
class Task:
    def __init__(self, id, length):
        self.id = id
        self.length = length

# Define MEC server with its VMs
class MECServer:
    def __init__(self, id, num_vms, vm_capabilities):
        self.id = id
        self.vms = [np.random.choice(vm_capabilities) for _ in range(num_vms)]
        print(f"MEC Server {self.id} VMs: {self.vms}")

# Create an MEC server
mec_server = MECServer(id=1, num_vms=num_vms, vm_capabilities=vm_capabilities)

# Function to calculate EPT for each task on each VM
def calculate_ept(task, vm_capability):
    return task.length / vm_capability

# Define the fitness function
def fitness_function(solution, ept_matrix, vm_capabilities):
    execution_time = np.max([np.sum(ept_matrix[:, j] * (solution == j)) for j in range(len(vm_capabilities))])
    energy_consumption = np.sum([np.sum(ept_matrix[:, j] * (solution == j)) * 0.65 for j in range(len(vm_capabilities))])
    b = 0.5  # Balancing parameter
    return (1 - b) * execution_time + b * energy_consumption

# Implement Improved Grey Wolf Optimizer (I-GWO)
def gwo_optimization(num_wolves, num_dimensions, max_iterations, fitness_function, ept_matrix, vm_capabilities):
    a = 2  # Coefficient

    # Initialize alpha, beta, and delta wolves
    alpha_wolf, beta_wolf, delta_wolf = np.zeros(num_dimensions), np.zeros(num_dimensions), np.zeros(num_dimensions)
    alpha_score, beta_score, delta_score = float('inf'), float('inf'), float('inf')

    wolves = np.random.randint(0, len(vm_capabilities), (num_wolves, num_dimensions))  # Initialize wolves

    for iteration in range(max_iterations):
        for i in range(num_wolves):
            fitness = fitness_function(wolves[i], ept_matrix, vm_capabilities)
            if fitness < alpha_score:
                alpha_score, alpha_wolf = fitness, wolves[i]
            elif fitness < beta_score:
                beta_score, beta_wolf = fitness, wolves[i]
            elif fitness < delta_score:
                delta_score, delta_wolf = fitness, wolves[i]

        a = 2 - iteration * (2 / max_iterations)  # Update the coefficient 'a'

        for i in range(num_wolves):
            for j in range(num_dimensions):
                r1, r2 = np.random.rand(), np.random.rand()
                A1, C1 = 2 * a * r1 - a, 2 * r2

                D_alpha = abs(C1 * alpha_wolf[j] - wolves[i][j])
                X1 = alpha_wolf[j] - A1 * D_alpha

                r1, r2 = np.random.rand(), np.random.rand()
                A2, C2 = 2 * a * r1 - a, 2 * r2

                D_beta = abs(C2 * beta_wolf[j] - wolves[i][j])
                X2 = beta_wolf[j] - A2 * D_beta

                r1, r2 = np.random.rand(), np.random.rand()
                A3, C3 = 2 * a * r1 - a, 2 * r2

                D_delta = abs(C3 * delta_wolf[j] - wolves[i][j])
                X3 = delta_wolf[j] - A3 * D_delta

                wolves[i][j] = (X1 + X2 + X3) / 3

    return alpha_wolf, alpha_score

# Implement Dimension Learning-based Hunting (DLH) Strategy
def dimension_learning_hunting(wolves, num_wolves, num_dimensions, alpha_wolf, fitness_function, ept_matrix, vm_capabilities):
    new_wolves = np.copy(wolves)

    for i in range(num_wolves):
        RS_i = np.linalg.norm(wolves[i] - alpha_wolf)
        neighbors = []

        for j in range(num_wolves):
            if np.linalg.norm(wolves[i] - wolves[j]) < RS_i:
                neighbors.append(wolves[j])

        if len(neighbors) > 0:
            random_neighbor = neighbors[np.random.randint(len(neighbors))]
            new_position = wolves[i] + np.random.rand() * (random_neighbor - wolves[i])
            new_fitness = fitness_function(new_position, ept_matrix, vm_capabilities)

            if new_fitness < fitness_function(wolves[i], ept_matrix, vm_capabilities):
                new_wolves[i] = new_position

    return new_wolves

# Task scheduling optimization
def task_scheduling(num_wolves, num_dimensions, max_iterations, ept_matrix, vm_capabilities):
    # Initialize population
    wolves = np.random.randint(0, len(vm_capabilities), (num_wolves, num_dimensions))

    alpha_wolf, alpha_score = gwo_optimization(num_wolves, num_dimensions, max_iterations, fitness_function, ept_matrix, vm_capabilities)

    for iteration in range(max_iterations):
        wolves = dimension_learning_hunting(wolves, num_wolves, num_dimensions, alpha_wolf, fitness_function, ept_matrix, vm_capabilities)
        alpha_wolf, alpha_score = gwo_optimization(num_wolves, num_dimensions, max_iterations, fitness_function, ept_matrix, vm_capabilities)

    return alpha_wolf, alpha_score

# Define the different numbers of tasks to be tested
num_tasks_list = [40, 80, 120, 160]

# Maximum iterations
max_iterations = 15

# Run the task scheduling optimization for different numbers of tasks
for num_tasks in num_tasks_list:
    start_time = time.time()
    
    task_lengths = np.random.randint(10, 510, size=num_tasks)
    tasks = [Task(id=i, length=task_lengths[i]) for i in range(num_tasks)]
    print(f"Tasks Lengths: {[task.length for task in tasks]}")
    
    ept_matrix = np.zeros((len(tasks), num_vms))
    for i, task in enumerate(tasks):
        for j, vm in enumerate(mec_server.vms):
            ept_matrix[i][j] = calculate_ept(task, vm)

    best_wolf, best_fitness = task_scheduling(num_wolves=30, num_dimensions=num_tasks, max_iterations=max_iterations, ept_matrix=ept_matrix, vm_capabilities=vm_capabilities)
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"Number of Tasks: {num_tasks}")
    print(f"Best Wolf: {best_wolf}")
    print(f"Best Fitness: {best_fitness}")
    print(f"Execution Time: {execution_time:.2f} seconds\n")


MEC Server 1 VMs: [100, 1000, 1000, 100, 100, 100, 1000, 100, 1000, 1000]
Tasks Lengths: [183, 456, 233, 31, 344, 190, 98, 26, 371, 435, 488, 307, 443, 441, 488, 105, 332, 429, 390, 83, 332, 270, 140, 455, 252, 200, 104, 110, 118, 46, 299, 227, 350, 277, 188, 273, 194, 95, 198, 62]
Number of Tasks: 40
Best Wolf: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
Best Fitness: 27.996474999999997
Execution Time: 3.96 seconds

Tasks Lengths: [450, 129, 471, 286, 123, 119, 366, 278, 266, 20, 448, 104, 101, 217, 253, 411, 61, 358, 183, 457, 34, 325, 273, 409, 284, 373, 222, 403, 438, 386, 22, 228, 36, 194, 418, 99, 449, 79, 228, 337, 123, 52, 93, 174, 343, 178, 348, 501, 212, 497, 55, 101, 252, 98, 252, 313, 339, 34, 282, 222, 132, 464, 46, 207, 83, 123, 394, 451, 205, 231, 231, 433, 476, 325, 468, 483, 443, 351, 325, 210]
Number of Tasks: 80
Best Wolf: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0